In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

# Two-stage conv net (heatmap + poses)

In [4]:
import tensorflow as tf
import cv2
import math

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.activations import *

mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    input_pose = Input(shape=(x_pose_train[0].shape[0],))
    x = Reshape(target_shape=(num_consec, x_pose_train[0].shape[0] // (2 * num_consec), 2, 1))(input_pose)
    x = Conv3D(
        filters=20,
        kernel_size=(1, 38, 2),
        kernel_initializer='he_uniform',
        activation='relu',
        data_format="channels_last",
    )(x)
    x_base = Reshape(target_shape=(num_consec, 20))(x)
    x2 = Conv1D(
        filters=10,
        kernel_size=2,
        kernel_initializer='he_uniform',
        activation='relu',
    )(x_base)
    x2 = Dense(
        6,
        activation='relu'
    )(Flatten()(x2))
    x3 = Conv1D(
        filters=8,
        kernel_size=3,
        kernel_initializer='he_uniform',
        activation='relu',
    )(x_base)
    x3 = Dense(
        4,
        activation='relu'
    )(Flatten()(x3))

    y_pose = Concatenate()([x2, x3])

    nfilter = 4
    
    shape = x_heatmap_train[0].shape
    input_heatmap = Input(shape=shape)
    x = Reshape(target_shape=(shape[0], shape[1], shape[2], 1))(input_heatmap)
    x2 = Conv3D(
        filters=nfilter,
        kernel_size=(5, 5, 5),
        kernel_initializer='he_uniform',
        activation='relu',
        data_format="channels_last",
    )(x)
    x2 = MaxPooling3D(
        pool_size=(1, 4, 4), strides=None, padding="valid"
    )(x2)
    x2 = BatchNormalization()(x2)
    x2 = Conv3D(
        filters=nfilter,
        kernel_size=(1, 3, 3),
        kernel_initializer='he_uniform',
        activation='relu',
        data_format="channels_last",
    )(x2)
    x2 = MaxPooling3D(
        pool_size=(1, 2, 2), strides=None, padding="valid"
    )(x2)
    x2 = BatchNormalization()(x2)
    x2 = Conv3D(
        filters=nfilter,
        kernel_size=(1, 6, 6),
        kernel_initializer='he_uniform',
        activation='relu',
        data_format="channels_last",
    )(x2)
    x2 = MaxPooling3D(
        pool_size=(1, 6, 6), strides=None, padding="valid"
    )(x2)
    x2 = BatchNormalization()(x2)
    
    x2 = Dense(
        4,
        activation='relu'
    )(Flatten()(x2))
    
    y_heatmap = x2

    y_c = Concatenate()([y_pose, y_heatmap])
    output_layer = Dense(3)(y_c)
    output_layer = Softmax()(output_layer)

    model = Model([input_pose, input_heatmap], output_layer)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task

In [5]:
model.compile(
    optimizer='adam', 
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 5, 288, 512) 0                                            
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 5, 288, 512,  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 1, 284, 508,  504         reshape_2[0][0]                  
__________________________________________________________________________________________________
max_pooling3d (MaxPooling3D)    (None, 1, 71, 127, 4 0           conv3d_1[0][0]                   
______________________________________________________________________________________________

In [6]:
import random

p = sum(y_train > 0) / sum(y_train == 0)
subsample = []
for i in range(y_train.shape[0]):
    if random.random() < p or y_train[i]:
        subsample.append(i)

x_pose_sub = x_pose_train[subsample]
x_heatmap_sub = x_heatmap_train[subsample]
y_sub = y_train[subsample]

from sklearn.model_selection import train_test_split
x_pose_tr, x_pose_val, x_heatmap_tr, x_heatmap_val, y_tr, y_val = train_test_split(
    x_pose_sub, x_heatmap_sub, y_sub, 
    test_size=0.05, shuffle=True
)

In [ ]:
# from tensorflow.keras import backend as K
# K.set_value(model.optimizer.learning_rate, 1e-4)

# with tf.device('/gpu:2'):
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

model.fit(
    [x_pose_tr, x_heatmap_tr], y_tr, 
    batch_size=8, 
    epochs=3000,
#     class_weight={0: 1, 1: 1, 2: 1, 3: 10},
    shuffle=True,
    validation_data=([x_pose_val, x_heatmap_val], y_val),
    verbose=1)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/3000
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
177/179 [============================>.] - ETA: 0s - loss: 1.0840 - accuracy: 0.4411

In [29]:
from sklearn.metrics import classification_report

y_pred = np.argmax(model.predict(x_val), axis=1)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      1293
           1       0.87      0.88      0.88       632
           2       0.90      0.89      0.90       638

    accuracy                           0.90      2563
   macro avg       0.90      0.90      0.90      2563
weighted avg       0.90      0.90      0.90      2563



In [30]:
os.chdir('/home/code-base/user_space/ai-badminton/TrackNetv2/3_in_3_out')

In [31]:
from tensorflow.python.keras.saving import hdf5_format
import h5py

with h5py.File('./hitnet_conv_model_predict_player_no_shuttle.h5', mode='w') as f:
    hdf5_format.save_model_to_hdf5(model, f)

# Create network to identify who hit the shuttle

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import math

from ai_badminton.trajectory import Trajectory
from ai_badminton.hit_detector import AdhocHitDetector, MLHitDetector
from ai_badminton.pose import Pose, read_player_poses, process_pose_file
from ai_badminton.court import Court, read_court
from ai_badminton.video_annotator import annotate_video

In [3]:
import os
import pandas as pd
import numpy as np
import cv2

def read_court(filename):
    file = open(filename, 'r')
    coordinates = [[float(x) for x in line.split(';')] for line in file]
    return coordinates

num_consec = 5
matches = list('match' + str(i) for i in range(1, 4))

from sklearn.preprocessing import MinMaxScaler, StandardScaler, minmax_scale

manual_label = {
    ('match1', '1_02_04'): 2,
    ('match1', '1_06_08'): 1,
    ('match1', '1_02_02'): 1,
    ('match1', '1_02_01'): 2,
    ('match1', '1_03_04'): 2,
    ('match1', '1_02_03'): 1,
    ('match3', '1_08_10'): 2,
    ('match8', '3_02_00'): 2,
}

x_pose_train, x_heatmap_train, y_train = [], [], []
for match in matches:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        if '.mp4' not in video:
            continue
        cap = cv2.VideoCapture('/home/code-base/scratch_space/data/%s/rally_video/%s' % (match, video))
        shape = cap.read()[1].shape
        height, width = shape[0], shape[1]

        rally, _ = os.path.splitext(video)
        trajectory = Trajectory('/home/code-base/scratch_space/data/%s/ball_trajectory/%s_ball.csv' % (match, rally))
        hit = pd.read_csv('/home/code-base/scratch_space/data/%s/shot/%s_hit.csv' % (match, rally))
        
        poses = read_player_poses('/home/code-base/scratch_space/data/%s/poses/%s' % (match, rally))
        bottom_player, top_player = poses[0], poses[1]
        
        x_pose_list, x_heatmap_list, y_list = [], [], []
        court_pts = read_court('/home/code-base/scratch_space/data/%s/court/%s.out' % (match, rally))
        corners = np.array([court_pts[1], court_pts[2], court_pts[0], court_pts[3]]).flatten()
    
        heatmaps = np.load('/home/code-base/scratch_space/data/%s/ball_trajectory/%s.npz' % (match, rally))
        heatmaps = heatmaps['arr_0']
        
        def reflect(x):
            x_ = np.array(x)
            for i in range(0, x.shape[1], 2):
                x_[:, i] = width - x_[:, i]
            return x_

        # Identify first hit by distance to pose
        # and then alternate hits
        def dist_to_pose(pose, p):
            pose = pose.reshape(17, 2)
            p = p.reshape(1, 2)
            D = np.sum((pose - p) * (pose - p), axis=1)
            return min(D)
        
        y_new = np.array(hit.hit.to_numpy())
        # Majority vote for who started the rally
        # If hit number is odd, then whoever started the rally
        # is the opposite of whoever was detected.
        votes = [0, 0]
        best_dist = [1e99, 1e99]
        nhit = 0
        for i in range(y_new.shape[0]):
            if not y_new[i]:
                continue
            
            p = np.array([trajectory.X[i], trajectory.Y[i]])
            db = dist_to_pose(bottom_player.values[i], p)
            dt = dist_to_pose(top_player.values[i], p)
            person = 0
            if db < dt:
                person = 1
            else:
                person = 2
            
            if nhit % 2:
                person = 3 - person
                
            votes[person - 1] += 1
            best_dist[person - 1] = min(best_dist[person - 1], min(db, dt))
            
            nhit += 1
 
        if abs(votes[0] - votes[1]) < 2:
            print(match, rally, 'is hard', votes, best_dist)
        else:
            print(match, rally, 1 if votes[0] > votes[1] else 2, votes)
            
        last = 2 if votes[0] > votes[1] else 1
        if (match, rally) in manual_label:
            last = 3 - manual_label[match, rally]
            print('Manual label applied.')
        for i in range(y_new.shape[0]):
            if not y_new[i]:
                continue
                
            y_new[i] = 3 - last
            last = y_new[i]

        for i in range(num_consec):
            end = min(
                len(trajectory.X), 
                len(hit.hit), 
                heatmaps.shape[0]
            ) - num_consec + i + 1
            x_heatmap = heatmaps[i:end]
            
            # Use entire pose
            x_pose = np.hstack([bottom_player.values[i:end], top_player.values[i:end]])
            x = np.hstack([x_pose, np.array([corners for j in range(i, end)])])

            y = y_new[i:end]
            x_pose_list.append(x)
            x_heatmap_list.append(x_heatmap)
            y_list.append(y)

        x_t = np.hstack(x_pose_list)
        x_ht = np.stack(x_heatmap_list, axis=1)
        y_t = np.max(np.column_stack(y_list[1:-1]), axis=1)
        
        del heatmaps
        del x_heatmap
        
        def scale_data(x):
            x = np.array(x)
            def scale_by_col(x, cols):
                x_ = x[:, cols]
                m, M = np.min(x_[x_ != 0]), np.max(x_[x_ != 0])
                x_[x_ != 0] = (x_[x_ != 0] - m) / (M - m) + 1
                x[:, cols] = x_
                return x
            
            even_cols = [2*i for i in range(x.shape[1] // 2)]
            odd_cols = [2*i+1 for i in range(x.shape[1] // 2)]
            x = scale_by_col(x, even_cols)
            x = scale_by_col(x, odd_cols)
            return x
        
        identity = lambda x: x
        augmentations = [identity]
        for transform in augmentations:
            x_pose_train.append(scale_data(transform(x_t)))
            x_heatmap_train.append(x_ht)
            y_train.append(y_t)

import gc

x_pose_train = np.vstack(x_pose_train)
x_heatmap_train = np.vstack(x_heatmap_train)
y_train = np.hstack(y_train)

gc.collect()

match1 1_02_00 1 [13, 5]
match1 1_02_04 is hard [1, 1] [7059.074806800006, 859.8963510596025]
Manual label applied.
match1 1_06_08 1 [4, 0]
Manual label applied.
match1 1_02_02 1 [5, 1]
Manual label applied.
match1 1_06_06 2 [0, 5]
match1 1_03_05 2 [3, 10]
match1 1_02_01 2 [0, 6]
Manual label applied.
match1 1_03_04 2 [0, 3]
Manual label applied.
match1 1_06_09 1 [11, 3]
match1 1_02_03 is hard [1, 0] [5729.763825145301, 1e+99]
Manual label applied.
match1 1_03_06 2 [3, 5]
match1 1_01_00 2 [1, 9]
match2 1_00_02 1 [17, 1]
match2 1_08_11 1 [6, 1]
match2 1_06_08 1 [7, 0]
match2 1_09_12 1 [11, 0]
match2 1_08_12 1 [5, 1]
match2 1_04_03 2 [1, 5]
match2 1_06_09 2 [0, 10]
match2 1_02_03 2 [0, 5]
match3 2_04_07 2 [6, 15]
match3 2_18_15 2 [0, 37]
match3 1_12_17 2 [3, 5]
match3 3_11_10 2 [0, 6]
match3 2_10_12 1 [5, 1]
match3 1_08_10 is hard [1, 2] [4583.862849142903, 1176.3588272543989]
Manual label applied.
match3 1_01_00 2 [0, 13]


308